In [1]:
from vpython import *

from astropy.time import Time
import numpy as np
from PIL import Image

<IPython.core.display.Javascript object>

# Definition of a Force Field

In [2]:
class ForceField(object):
    def __init__(self,name):
        self.name = name
    '''
    return the acceleration vector given position r
    '''
    def a(self,r):
        pass

## Define the Force Field of Earth In the Case of spherical Earth Model
$$
    F = - \frac{GM}{r^2}\frac{\boldsymbol{r}}{r}
$$


In [3]:
class GravitationalForceField(ForceField):
    def __init__(self):
        self.GM = 398600.4405e9
        self.R = 6371000
    
    def a(self,r): # r = [x,y,z]
        rr = np.sqrt(np.sum(r ** 2))
        
        return -self.GM/(rr ** 3) *r

## In the case of Flattened Earth The Force Field includes additional terms (J2)


The potential with respect to a flattened earth is defined as:

$$
U(r) = \frac{GM}{r} - \frac{J_2 G M R^2}{2r^3} \left ( 3 \left (\frac{z}{r} \right )^2 - 1 \right )
$$


The gravitational acceleration is given as the $\nabla U$.

$$
\nabla U(r) = \begin{bmatrix}
\frac{\partial{U}}{\partial{x}} \\
\frac{\partial{U}}{\partial{y}} \\
\frac{\partial{U}}{\partial{z}}
\end{bmatrix}
=
\begin{bmatrix}
-\frac{G M x}{r^3} -\frac{1}{2}J_2 G M R^2 \left (\frac{3x}{r^5} - \frac{15xz^2}{r^7}\right )\\
-\frac{G M y}{r^3} -\frac{1}{2}J_2 G M R^2 \left (\frac{3y}{r^5} - \frac{15yz^2}{r^7}\right ) \\
-\frac{G M z}{r^3} -\frac{1}{2}J_2 G M R^2 \left (\frac{9z}{r^5} - \frac{15z^3}{r^7}\right )
\end{bmatrix}
$$

Given
$$
GM = 398600.4405x10^9
$$

$$
J_2 =  1.0826157x10^{-3}
$$

$$
R = 6378135
$$



In [4]:
class ForceFieldEllipsoidalEarth(GravitationalForceField):
    def __init__(self):
        self.GM = 398600.4405e9
        self.R = 6378135
        self.J2 = 1.0826157e-3
        
    def a(self, r):
        a_sphere = GravitationalForceField.a(self,r)
        
        a_flat = np.array([self.partial_x(r), self.partial_y(r), self.partial_z(r)])
        
        return a_sphere + a_flat
    
    def partial_x(self, rad):
        r = np.sqrt(np.sum(rad**2))
        x  = rad[0]
        y = rad[1]
        z = rad[2]

       #rad = np.sqrt(r[0]**2 + r[1]**2 + r[2]**2 )
        
        return -0.5*self.GM*self.J2*self.R**2 * (3*x/(r**5) - 15*x*(z**2)/(r**7))
    
    def partial_y(self, rad):
        r = np.sqrt(np.sum(rad**2))
        x  = rad[0]
        y = rad[1]
        z = rad[2]

        return -0.5*self.GM*self.J2*self.R**2 * (3*y/(r**5) - 15*y*(z**2)/(r**7))

    
    def partial_z(self, rad):
        r = np.sqrt(np.sum(rad**2))
        x  = rad[0]
        y = rad[1]
        z = rad[2]

        return -0.5*self.GM*self.J2*self.R**2 * (9*z/(r**5) - 15*(z**3)/(r**7))     

In [5]:
# Simple Euler integration
class Integrator(object):
    def __init__(self):
        pass
    
    def initialize(self, state):
        self.state = state
        
    ##
    # Calculate next state given ODE
    #
    def next(self, t, dt):
        s = self.state.s(t)
        ds = self.state.ds(t,dt, s)
        return s+ dt*ds
    

#
# Runge Kutta Integration
# see http://www.foo.be/docs-free/Numerical_Recipe_In_C/c16-1.pdf
#
class RK2Integrator(Integrator):
    def __init__(self):
        pass
        
    def next(self, t, dt):
        s = self.state.s(t)
        
        ## Find full step ahead
        k1 = dt * self.state.ds(t,dt,s)
        
        ## use half of the full step ahead for correction
        k2 = dt*self.state.ds(t,dt/2.0, s+k1/2.0)
        
        return s+k2


class SatelliteState(object):
    ## Initialize a satellite state with initial position,
    ## initial velocity
    ## the geopotential force field
    ## and a numerical integrator
    def __init__(self, i_pos, i_vel, force_field, integ):
        self.position = np.array(i_pos)
        self.velocity = np.array(i_vel)
        self.force_field = force_field
        self.integrator = integ
        ## Intialize the integrator with the initial state of pos, and velocity
        self.integrator.initialize(self)
        
        
    
    @classmethod
    def fromKeplerElements(cls, force_field, integ, a, e, incl, theta, omega, T, t):
        GM = force_field.GM
        
        incl = np.radians(incl)
        theta = np.radians(theta)
        omega = np.radians(omega)
        
        M = np.sqrt(GM/(a**3)) * (t-T)
        
        E = M
        
        for i in range(3):
            E = M + e*np.sin(E)
        
        v = 2*np.arctan( np.sqrt((1+e)/(1-e)) * np.tan(E/2.0))
        r = a*(1-e*np.cos(E))
        
        R3_omega = np.array([
            [np.cos(omega), -np.sin(omega), 0],
            [np.sin(omega), np.cos(omega), 0],
            [0,             0,              1]
            
        ])

        R3_theta = np.array([
            [np.cos(theta), -np.sin(theta), 0],
            [np.sin(theta), np.cos(theta), 0],
            [0,             0,              1]
            
        ])

        R1_i = np.array([
            [1,             0,              1],
            [0, np.cos(incl), -np.sin(incl)],
            [0, np.sin(incl), np.cos(incl)],
            
        ])


        x_sat = np.array([r*np.cos(v), r*np.sin(v), 0])
        
        x_sat = np.dot(R3_omega, np.dot( R1_i, np.dot(R3_theta, x_sat)))
        
        p = a*(1-e**2)
        
        v_sat = np.array([-np.sqrt(GM/p)*np.sin(v), np.sqrt(GM/p)*(e+np.cos(v)), 0])
        
        v_sat = np.dot(R3_omega, np.dot( R1_i, np.dot(R3_theta, v_sat)))
        
        return SatelliteState(x_sat, v_sat, force_field, integ)
        
        
    
    '''
    Find and update the position in delta t seconds 
    Using the integrator
    '''
    def next(self,t, dt):
        ns = self.integrator.next(t,dt)
        self.position = np.copy(ns[:3])
        self.velocity = np.copy(ns[3:])
    
    # State at time t
    def s(self, t):
        ## state is position, velocity represented as a vector
        ## the first three items are x,y,z of satellite.
        ## the rest is the vx, vy, vz
        return np.hstack( (self.position, self.velocity) )
    
    ## derivative of state at time t
    def ds(self ,t, dt, s):
        ## First component is velocity, second component is gravitational acceleration
        ## calculated from potential field
        ## see gravity potential and its gradient
        return np.hstack( (s[3:],self.force_field.a(s[:3]) ) )


In [6]:
#
# A Class to represent a satellite object.

class Satellite(object):
    def __init__(self,state):
        self.state = state
        # this defines a sphere to represent satellite in
        # vpython 3d environment
        self.obj = sphere ( pos=vector(state.position[0],state.position[1],state.position[2]), 
                            radius = 100000, visible=True, color=vector(1,0,0),
                            make_trail=True, trail_type="curve", interval=1, retain=5000
                          )

    ## Find the next position 
    # Here we make an iteration with two seconds.
    #
    def next(self,t, dt):
        
        ## The number of two seconds iteration
        loop_count = dt // 2
        # Remaining sub two seconds
        remaining = dt % 2
        
        # Iterate for number of two seconds steps
        for i in range(int(loop_count)):
            self.state.next(t,2)
        # iterate for remaining two seconds
        self.state.next(t, remaining)
        
        # update satellite position
        # which is a 3d object in vpython
        self.obj.pos = vector(self.state.position[0],self.state.position[1],self.state.position[2])


In [7]:
# This class represents 3D model of earth for visual simulation of earth rotation
# GCRF and ITRF axes are also shown.

class Earth(object):
    def __init__(self):
        self.obj = sphere ( pos=vector(0,0,0),
                radius = 6371000, # radius of earth 
                visible=True, 
                texture=textures.earth,up=vector(0,0,1),
                flipx = False , shininess = 0.9,opacity=0.9)
        
        self.obj.rotate(angle=np.radians(90),axis=vector(0,0,1)) # X axis points to greenwhich meridian
        
        # The gravitational force field of a spherical earth
        self.force_field = ForceFieldEllipsoidalEarth()
        
        self.w = 72.92115e-6 # radians/sec The angular velocity of earth
        
        #The GCRF axes.
        sw = 100000
        hw=300000       
        self.arrow_z_GCRF = arrow(pos=vector(0,0,0),
                             axis=vector(0,0,6371000+800000), shaftwidth=sw,headwidth=hw,color=vector(0,0,1))


        self.arrow_y_GCRF = arrow(pos=vector(0,0,0),
                             axis=vector(0,6371000+800000,0),shaftwidth=sw,headwidth=hw,color=vector(0,1,0))

        self.arrow_x_GCRF = arrow(pos=vector(0,0,0),
                             axis=vector(6371000+5800000,0,0), shaftwidth=sw,headwidth=hw,color=vector(1,0,0))
        self.veq = text(text='Vernel Equinox',billboard=True, 
                        align='center', color=vector(1,1,0), 
                        pos = vector(6371000+5800000,0,0),height=10*sw)
        self.ncp = text(text='North CP',billboard=True, 
                align='center', color=vector(1,1,0), 
                pos = vector(0,0,6371000+800000),height=10*sw)


    
    def next(self, t, dt):

        delta_gast = self.calculate_EOP(t,dt)
        self.obj.rotate(angle = np.radians(delta_gast), axis=vector(0,0,1))
        
    def calculate_EOP(self, t, dt):
        return np.degrees(self.w*dt)
    
    
    
    def fromGCRFToECEF(self, pos, t):
        ## Convert the given position in GCRF at specified datetime t to ECEF
        return pos
    
    
    def fromECEFToGeodetic(self, pos, t):
        # Convert from ECEF to geodetic lat lon height
        return pos

In [ ]:
scene.width = 800
scene.height = 600
scene.title = "Earth Rotation Sample"
scene.visible=False
running = False
scene.caption = """Change the delta t to make simulation run faster. 
<br>To rotate "camera", drag with right button or Ctrl-drag.
<br>To zoom, drag with middle button or Alt/Option depressed, or use scroll wheel.
  On a two-button mouse, middle is left + right.
<br>To pan left/right and up/down, Shift-drag.
Touch screen: pinch/extend to zoom, swipe or two-finger rotate."""

t = Time('2012-11-20', scale='utc',location=('0','0'))

def Run(b):
    global running
    running = not running
    if running: b.text = "Pause"
    else: b.text = "Run"


        
button(text="Run", pos=scene.title_anchor, bind=Run)


def setspeed(s):
    wt.text = '{:1.2f}'.format(s.value)
    
sl = slider(min=1, max=1440, value=1, length=220, bind=setspeed, right=15)

wt = wtext(text='{:1.2f}'.format(sl.value))

# Load Vpython textures for earth
scene.waitfor("textures")
scene.visible = True  # show everything


# Time increment of simulation
# we will calculate satellite position with dt increments and redraw the scene.
dt = 1

# Create earth object 
# and also create the spherical gravity field around earth

earth = Earth()

sats = [] # an empty list of satellites to simulate

own_sat = Satellite(SatelliteState.fromKeplerElements(
    earth.force_field, RK2Integrator(),6371000+1600000,
                                                     0.001,
                                                     20,
                                                      0,
                                                      0,
                                                      0,
                                                      0
                                                     ))

sats.append(own_sat)


# Siulation loop

while True:
    # limit Frame per second
    rate(30)
    
    if (not running):
        continue
    
    # Adjust delta t of simulation
    dt = sl.value
    # Adjust time
    
    t = t + (dt/86400.0)
    ## And print time in modified julian days
    wt.text = ' %s dt = %d seconds  JD=%f' % (str(t),sl.value,t.mjd)

    ## Find next positions of satellites 
    for sat in sats:
        sat.next(t,dt)
    ## Rotate earth with angular speed
    earth.next(t, dt)    

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>